In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt 
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

***A student performance dataset containing their details along with the scores of maths,
  reading and writing of their recent exam***

# 1. DATA CLEANING:

In [ ]:
# we read the file and change all missing values to NaN
v=['NA','none','missing']
data=pd.read_csv('../input/studentsperformance12th/StudentsPerformance.csv',na_values=v)

In [ ]:
#checking the data types and the count of non-null values
data.info()
#we could observe there are 1004 rows and 8 columns having 5 string type and 4 float type

In [ ]:
data.describe()

In [ ]:
#Here we can observe that some entries contain NaN
pd.set_option("max_rows", None)
data

In [ ]:
#we drop all the entries having NaN in all their columns
data.dropna(axis='index',how='all',inplace=True)
data

#Here we observe that 3,142,537 AND 931th entries have been dropped

In [ ]:
#Now we check the dataset after removing empty entries

data.info()

#There are 1000 entries and the first 4 columns have no null values.

In [ ]:
# We assume NaN in scores represent student is absent for exam and
# change the NaN values in scores to 0.0
data[['math score','reading score','writing score']]=data[['math score','reading score','writing score']].fillna(value=0.0)

In [ ]:
# All NaN in test preparation course represent non-completed 
data['test preparation course']=data['test preparation course'].fillna('non completed')

In [ ]:
#The dataset is completly cleaned without any NaN values
data.info()

In [ ]:
#Lets check no.of duplicates and then drop them
data.duplicated().sum()
data.drop_duplicates(inplace=True)

In [ ]:
#we are incrementing the index by 1.
data.index=data.index+1

In [ ]:
#naming index column to roll_no
data.index.name = 'roll_no'

> # **2.DATA MANIPULATION**

* **Finding gpa for the three subjects**

In [ ]:
#lets get the gpa of each student
data['gpa'] = data[['math score', 'reading score','writing score']].mean(axis=1)

In [ ]:
#Converting the float-GPA to int-GPA for better analysis
data['gpa']=data['gpa'].astype(int)

* **Adding a column for pass/fail based on the gpa**

In [ ]:
#If student has gpa>40, its pass. Otherwise its fail
col = np.where(data["gpa"]>=40, True, False)
data['pass']=col
data['pass'].value_counts()

* **Assigning grades based on gpa**

In [ ]:
data.loc[(data['gpa']>=90)&(data['gpa']<=100), 'grade'] = 'S'
data.loc[(data['gpa']>=80)&(data['gpa']<90), 'grade'] = 'A'
data.loc[(data['gpa']>=70)&(data['gpa']<80), 'grade'] = 'B'
data.loc[(data['gpa']>=60)&(data['gpa']<70), 'grade'] = 'C'
data.loc[(data['gpa']>=50)&(data['gpa']<60), 'grade'] = 'D'
data.loc[(data['gpa']>=40)&(data['gpa']<50), 'grade']  = 'E'
data.loc[data['gpa']<40, 'grade'] = 'F'

* **A column for re-exam for any subject**

In [ ]:
#For maths- m, reading-r,writing-w, else- n
data['re_exam']='no'
data.loc[data['math score']==0.0,'re_exam']='m'
data.loc[data['reading score']==0.0,'re_exam']='r'
data.loc[data['writing score']==0.0,'re_exam']='w'

* **Final dataset** 

In [ ]:
pd.set_option("max_rows",None)
data

In [ ]:
#saving to our devices
data.to_csv('Final_student_sheet',index=True)

> # **3.DATA ANALYSIS**

* **Memory required in bytes for dataset**

In [ ]:
data.memory_usage()

* **The topper details of each subject**

In [ ]:
#MATHS
data.loc[data['math score']==100.0]

In [ ]:
#READING
data.loc[data['reading score']==100.0]

In [ ]:
#WRITING
data.loc[data['writing score']==100.0]

* **Student with highest gpa**

In [ ]:
data.nlargest(3,'gpa')

* **Sorting section-wise gpa and grades**

In [ ]:
data[['section','gpa','grade']].sort_values(by=['section','gpa','grade'])

* **Students who has re-exam in each subject**

In [ ]:
# MATHS
data.groupby('re_exam').get_group('m')

In [ ]:
#READING
data.groupby('re_exam').get_group('r')

In [ ]:
#WRITING
data.groupby('re_exam').get_group('w')

* **Details of students who failed**

In [ ]:
data['pass'].value_counts()

In [ ]:
data.query('0<= gpa <40')

* **The count of students in entire standard**

In [ ]:
data['gender'].value_counts()

Here, we can observe there are 517 female and 483 male students

# **4.DATA VISUALISATION**

* **The number of male and female student in each section:**

In [ ]:
sns.countplot(x='section', data = data, hue='gender')
plt.title('male to female count')
plt.show

Group C has the highest count of students with more men. Group D and Group E have almost equal ratio of both gender. In group A and B, the ratio highly differs.

* **A plot to represent the count of different level of parent's education:**

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(x ='parental level of education', data = data)
plt.show()

Here, most of the parents havent got a college degree and the secondmost count shows associate's degree,followed by completion of high school. So almost all the parents would have the basic educational knowledge to guide their children.

* **Which subject has highest mean score**

In [ ]:
a=data['math score'].mean()
b=data['reading score'].mean()
c=data['writing score'].mean()
print(f"maths:{a},reading:{b},writing:{c}")
average=[a,b,c]
score=['math score','reading score','writing score']
separate = (0,0.1,0) 
colors=['red','black','yellow']
fig = plt.figure(figsize =(10, 7))
plt.pie(average, labels = score,explode=separate,colors=colors,wedgeprops={'edgecolor':'black','linewidth':2,'linestyle':'solid'} )
plt.title("Average score of each subject")
plt.show()

Here, the students have scored more in reading while compared to maths and writing. There is only a small difference in average of the three subjects.

* **Count of students completed/not-completed test in each section**

In [ ]:
plt.figure(figsize=(12,4))
sns.countplot(x='section', data = data, hue='test preparation course', palette='bright')
plt.show

Most of the students in all the sections havent completed the test preparation course

* **Section wise gpa range**

In [ ]:
plt.figure(figsize=(15,10))
sns.catplot(x='section',y='gpa',data=data,saturation=1,kind="box",aspect=1,linewidth=1,height=8)     


students of group E has the highest gpa range followed by group D

* **The relation of grades to each subject:**

> **MATHS**

In [ ]:
plt.figure(figsize=(10,12))
ax=sns.catplot(data=data, kind="violin", x="grade", y="math score", split=True).set(title='Grades corresponding to maths score')


> **READING**

In [ ]:
plt.figure(figsize=(10,12))
sns.catplot(data=data, kind="violin", x="grade", y="reading score", split=True).set(title='Grades corresponding to reading score')

> **WRITING**

In [ ]:
plt.figure(figsize=(10,12))
sns.catplot(data=data, kind="violin", x="grade", y="writing score", split=True).set(title='Grades corresponding to writing score')

here, we could observe even if one scores high in any one of the subject, the grades scored is low due to the marks in others. Students with high scores in all subjects got higher grade.

* **Grades proportion of students**

In [ ]:
values=data['grade'].value_counts()
separate = (0,0,0,0,0,0,0.3) 
x=['S','A','B','C','D','E','F']
fig = plt.figure(figsize =(10, 7))
plt.pie(values, labels = x,explode=separate,wedgeprops={'edgecolor':'black','linewidth':2,'linestyle':'solid','antialiased':True})
plt.title("Grades of students")
plt.show()

The percentage of low grades are less as compared to higher grades